### Reading json data with an inferred schema

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (SparkSession.builder
         .appName("read-xml-data")
         .config('spark.jars.packages', 'com.databricks:spark-xml_2.12:0.16.0')
         .master("spark://spark-master:7077")
         .config("spark.executor.memory", "512m")
         .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-94f2c32c-225c-42cc-a314-6c5fede1e3b2;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.16.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
downloading https://repo1.maven.org/maven2/com/databricks/spark-xml_2.12/0.16.0/spark-xml_2.12-0.16.0.jar ...
	[SUCCESSFUL ] com.databricks#spark-xml_2.12;0.16.0!spark-xml_2.12.jar (260ms)
downloading https://repo1.maven.org/maven2/commons-io/commons-io/2.11.0/commons-io-2.11.0.jar ...
	[SUCCESSFUL ] commons-io#commons-io;2.11.0!commons-io.jar (229ms)
downloading https://repo1.maven.org/maven2/org/glassfish/jaxb/txw2/3.0.2/txw2-3.0.

In [ ]:
# Read XML file into a DataFrame
df = (spark.read.format("com.databricks.spark.xml")
      .option("rowTag", "row")
      .load("../data/nobel_prizes.xml"))

In [3]:
df.printSchema()

root
 |-- category: string (nullable = true)
 |-- laureates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- firstname: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- motivation: string (nullable = true)
 |    |    |-- share: long (nullable = true)
 |    |    |-- surname: string (nullable = true)
 |-- overallMotivation: string (nullable = true)
 |-- year: long (nullable = true)



In [4]:
# Display contents of DataFrame
df.show()

# Alternatively

# df.show(50)  # Display first 50 rows
# df.show(10, truncate=False)  # Display first 10 rows without truncation

+----------+--------------------+--------------------+----+
|  category|           laureates|   overallMotivation|year|
+----------+--------------------+--------------------+----+
| chemistry|[{Carolyn, 1015, ...|                null|2022|
| economics|[{Ben, 1021, "for...|                null|2022|
|literature|[{Annie, 1017, "f...|                null|2022|
|     peace|[{Ales, 1018, "Th...|                null|2022|
|   physics|[{Alain, 1012, "f...|                null|2022|
|  medicine|[{Svante, 1011, "...|                null|2022|
| chemistry|[{Benjamin, 1002,...|                null|2021|
| economics|[{David, 1007, "f...|                null|2021|
|literature|[{Abdulrazak, 100...|                null|2021|
|     peace|[{Maria, 1005, "f...|                null|2021|
|   physics|[{Syukuro, 999, "...|"for groundbreaki...|2021|
|  medicine|[{David, 997, "fo...|                null|2021|
| chemistry|[{Emmanuelle, 991...|                null|2020|
| economics|[{Paul, 995, "for...|       

In [5]:
df.select("category", "year").show()

+----------+----+
|  category|year|
+----------+----+
| chemistry|2022|
| economics|2022|
|literature|2022|
|     peace|2022|
|   physics|2022|
|  medicine|2022|
| chemistry|2021|
| economics|2021|
|literature|2021|
|     peace|2021|
|   physics|2021|
|  medicine|2021|
| chemistry|2020|
| economics|2020|
|literature|2020|
|     peace|2020|
|   physics|2020|
|  medicine|2020|
| chemistry|2019|
| economics|2019|
+----------+----+
only showing top 20 rows



In [6]:
(df.select("category", "year"
     , col("laureates").getItem(0).id).show())

+----------+----+---------------+
|  category|year|laureates[0].id|
+----------+----+---------------+
| chemistry|2022|           1015|
| economics|2022|           1021|
|literature|2022|           1017|
|     peace|2022|           1018|
|   physics|2022|           1012|
|  medicine|2022|           1011|
| chemistry|2021|           1002|
| economics|2021|           1007|
|literature|2021|           1004|
|     peace|2021|           1005|
|   physics|2021|            999|
|  medicine|2021|            997|
| chemistry|2020|            991|
| economics|2020|            995|
|literature|2020|            993|
|     peace|2020|            994|
|   physics|2020|            988|
|  medicine|2020|            985|
| chemistry|2019|            976|
| economics|2019|            982|
+----------+----+---------------+
only showing top 20 rows



In [8]:
df_flattened = (
    df
    .withColumn("laureates",explode(col("laureates")))
    .select(col("category")
            , col("year")
            , col("overallMotivation")
            , col("laureates.id")
            , col("laureates.firstname")
            , col("laureates.surname")
            , col("laureates.share")
            , col("laureates.motivation")))

df_flattened.show(truncate=False)

+----------+----+-----------------+----+--------------------------+-----------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|category  |year|overallMotivation|id  |firstname                 |surname    |share|motivation                                                                                                                                                                                                                                                                                                                                                                              |
+----------+----+-----------------+----+--------------------------+-------

In [9]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType(
    [StructField('category', StringType(), True), 
     StructField('laureates', ArrayType(StructType(
         [StructField('firstname', StringType(), True),
          StructField('id', StringType(), True), 
          StructField('motivation', StringType(), True), 
          StructField('share', StringType(), True), 
          StructField('surname', StringType(), True)]), True), True),
     StructField('overallMotivation', StringType(), True),
     StructField('year', IntegerType(), True)])

# Read XML file into a DataFrame
df_with_schema = (spark.read.format("com.databricks.spark.xml")
                  .schema(schema)
                  .option("rowTag", "row")
                  .load("../data/nobel_prizes.xml"))

df_with_schema.show()

+----------+--------------------+--------------------+----+
|  category|           laureates|   overallMotivation|year|
+----------+--------------------+--------------------+----+
| chemistry|[{Carolyn, 1015, ...|                null|2022|
| economics|[{Ben, 1021, "for...|                null|2022|
|literature|[{Annie, 1017, "f...|                null|2022|
|     peace|[{Ales, 1018, "Th...|                null|2022|
|   physics|[{Alain, 1012, "f...|                null|2022|
|  medicine|[{Svante, 1011, "...|                null|2022|
| chemistry|[{Benjamin, 1002,...|                null|2021|
| economics|[{David, 1007, "f...|                null|2021|
|literature|[{Abdulrazak, 100...|                null|2021|
|     peace|[{Maria, 1005, "f...|                null|2021|
|   physics|[{Syukuro, 999, "...|"for groundbreaki...|2021|
|  medicine|[{David, 997, "fo...|                null|2021|
| chemistry|[{Emmanuelle, 991...|                null|2020|
| economics|[{Paul, 995, "for...|       

In [10]:
spark.stop() 